In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.CodeMirror {font-family:Consolas; font-size:10pt;}
div.output {font-size:10pt;}
div.input {font-family:Consolas; font-size:10pt;}
div.prompt {min-width:70px;}
</style>
"""))

**<font color='darkred' size='6'>ch03. 연관분석(장바구니분석)</font>**

# 연관분석 개요 

<pre>
- 데이터들 사이에서 <b>'자주'</b> 발생하는 속성을 찾고 그 속성들 사이에 어느 정도 있는지를 분석하는 방법 
- 활용분야: 상품진열, 사기보험적발, 카탈로그디자인, 신상품 카테고리 구성 
- 연관성분석 관련 지표 
    - 지지도(support) = 조건결과 항목 수/ 전체 수
    - 신뢰도(confidence) = 조건결과 항목 수 / 조건 항목 수 
    - 향상도(lift): 우연히 발생한 규칙인지 아닌지 여부를 확인 
                   = 조건 결과 지지도 / (조건지지도) * (결과지지도) 
                       => 1(상관관계 없음) 
                         >1(양의 상관관계) 
                         <1(음의 상관관계)
                   
    [조건] => [결과]  지지도   신뢰도    향상도  (cf_basket.csv) 
    [주스] => [콜라]   0.4       1      0.4 / (0.4) * (1) = 1   
    [소주] => [맥주]   0.2     0.333..  0.3 / (0.6) * (0.4) = 0.833...
</pre>

In [2]:
import os
os.getcwd() # 가져올 데이터는 반드시 이 폴더 안에 있어야 한다.

'D:\\BigData\\src\\09_자연어처리'

In [20]:
# 트랜잭션 데이터 가져오기 
import csv

transaction = []
with open('cf_basket.csv', 'r', encoding='utf-8') as cf:
    csvdata = csv.reader(cf)
    # transaction = list(csvdata) # 리스트로 형변환
    for row in csvdata:           # 혹은 for문 돌리기
        transaction.append(row)

FileNotFoundError: [Errno 2] No such file or directory: 'cf_basket.csv'

In [4]:
transaction

[['소주', '콜라', '와인'],
 ['소주', '오렌지주스', '콜라'],
 ['콜라', '맥주', '와인'],
 ['소주', '콜라', '맥주'],
 ['오렌지주스', '와인', '콜라']]

# 연관분석
- pip install apyori

## 연관 규칙 생성 

In [4]:
from apyori import apriori # 연관성 규칙 생성 

rules = apriori(transaction, min_support=0.2, min_confidence=0.1) # 지지도가 0.2이상인 경우만 연관분석해라.. .. 
rules = list(rules)
len(rules)

0

In [6]:
rules[15]

RelationRecord(items=frozenset({'콜라', '와인', '맥주'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'콜라', '맥주', '와인'}), confidence=0.2, lift=1.0), OrderedStatistic(items_base=frozenset({'맥주'}), items_add=frozenset({'콜라', '와인'}), confidence=0.5, lift=0.8333333333333334), OrderedStatistic(items_base=frozenset({'와인'}), items_add=frozenset({'콜라', '맥주'}), confidence=0.33333333333333337, lift=0.8333333333333334), OrderedStatistic(items_base=frozenset({'콜라'}), items_add=frozenset({'맥주', '와인'}), confidence=0.2, lift=1.0), OrderedStatistic(items_base=frozenset({'와인', '맥주'}), items_add=frozenset({'콜라'}), confidence=1.0, lift=1.0), OrderedStatistic(items_base=frozenset({'콜라', '맥주'}), items_add=frozenset({'와인'}), confidence=0.5, lift=0.8333333333333334), OrderedStatistic(items_base=frozenset({'콜라', '와인'}), items_add=frozenset({'맥주'}), confidence=0.33333333333333337, lift=0.8333333333333334)])

In [7]:
rules[1]

RelationRecord(items=frozenset({'소주'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'소주'}), confidence=0.6, lift=1.0)])

In [8]:
print('조건    ->     결과 \t 지지도 \t 신뢰도 \t 향상도')
for row in rules:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ', '.join(x for x in item[0])
        lhs = [x for x in item[0]] # 조건 (left hand side)
        rhs = ', '.join(x for x in item[1])
        rhs = [x for x in item[1]] # 결과 (right hand side)
        confidence = item[2]
        lift = item[3]
        if lift != 1:
            print(lhs, '=>', rhs, '\t{:5.3f}\t{:5.3f}\t{:5.3f}'.format(support, confidence, lift))

조건    ->     결과 	 지지도 	 신뢰도 	 향상도
['맥주'] => ['소주'] 	0.200	0.500	0.833
['소주'] => ['맥주'] 	0.200	0.333	0.833
['맥주'] => ['와인'] 	0.200	0.500	0.833
['와인'] => ['맥주'] 	0.200	0.333	0.833
['소주'] => ['오렌지주스'] 	0.200	0.333	0.833
['오렌지주스'] => ['소주'] 	0.200	0.500	0.833
['소주'] => ['와인'] 	0.200	0.333	0.556
['와인'] => ['소주'] 	0.200	0.333	0.556
['오렌지주스'] => ['와인'] 	0.200	0.500	0.833
['와인'] => ['오렌지주스'] 	0.200	0.333	0.833
['맥주'] => ['소주', '콜라'] 	0.200	0.500	0.833
['소주'] => ['콜라', '맥주'] 	0.200	0.333	0.833
['콜라', '맥주'] => ['소주'] 	0.200	0.500	0.833
['소주', '콜라'] => ['맥주'] 	0.200	0.333	0.833
['맥주'] => ['콜라', '와인'] 	0.200	0.500	0.833
['와인'] => ['콜라', '맥주'] 	0.200	0.333	0.833
['콜라', '맥주'] => ['와인'] 	0.200	0.500	0.833
['콜라', '와인'] => ['맥주'] 	0.200	0.333	0.833
['소주'] => ['콜라', '오렌지주스'] 	0.200	0.333	0.833
['오렌지주스'] => ['소주', '콜라'] 	0.200	0.500	0.833
['소주', '콜라'] => ['오렌지주스'] 	0.200	0.333	0.833
['콜라', '오렌지주스'] => ['소주'] 	0.200	0.500	0.833
['소주'] => ['콜라', '와인'] 	0.200	0.333	0.556
['와인'] => ['소주', '콜라'] 	0.200	0.333	

# 뉴스기사 연관 분석 실습 

## 뉴스 RSS을 이용해서 기사 검색 후 연관 분석 

In [5]:
import requests 
from bs4 import BeautifulSoup 

rss_url = "https://rss.joins.com/joins_money_list.xml"
money_response = requests.get(rss_url)
money_soup = BeautifulSoup(money_response.content, 'xml')
link_list = money_soup.select('item > link')
link_list = [l.text for l in link_list]
len(link_list)

30

In [15]:
from konlpy.tag import Kkma
kkma = Kkma()
news = []

for link in link_list:
    news_response =  requests.get(link) # (위의 리스트 컴프리헨션에서 .text를 포함하므로 여기서는 생략한다.)
    news_soup = BeautifulSoup(news_response.content, 'html.parser')
    news_content = news_soup.select_one('div#article_body').text # 기사 내용 
    nouns_list = list(filter(lambda word : len(word)>1, kkma.nouns(news_content))) # 1글자 넘을 때만 명사 추출해
    news.append(nouns_list)
news

[['마일리지',
  '통합',
  '결제',
  '플랫폼',
  '마일',
  '마일벌스',
  '벌스',
  '대표',
  '대표이사',
  '이사',
  '정진',
  '정진형',
  '26',
  '26일',
  '서비스',
  '중인',
  '디지털',
  '디지털상품권',
  '상품권',
  '10',
  '10여종',
  '여종',
  '구매',
  '구매상품',
  '상품',
  '추가',
  '업데이트',
  '이번',
  '상품추가',
  '기존',
  '판매',
  '30',
  '30여종',
  '편의점',
  '커피',
  '치킨',
  '베이커리',
  '영화',
  '뷰티',
  '이외',
  '게임',
  '기프트카드',
  '프트',
  '카드',
  '백화점',
  '외식',
  '가능',
  '사용자',
  '선택',
  '의미',
  '활성화',
  '도움',
  '예상',
  '신규',
  '유입',
  '효과',
  '전망',
  '제한',
  '사용처',
  '사용',
  '고객',
  '다양',
  '이용',
  '가맹점',
  '확장',
  '마일벌',
  '기업',
  '포인트',
  '교환',
  '커피전문점',
  '전문점',
  '실생활',
  '현금',
  '중앙',
  '중앙일보',
  '일보',
  '엠앤서비스',
  '제휴',
  '지원',
  '구글플레이',
  '플레이',
  '앱스토어',
  '다운로드',
  '당일',
  '당일기준',
  '기준',
  '출시',
  '60',
  '60일만',
  '누적',
  '30,000',
  '달성',
  '온라인'],
 ['한국',
  '한국과학기술정보',
  '과학',
  '기술',
  '정보',
  '연구원',
  '원장',
  '김재수',
  '재수',
  '이하',
  '프린팅',
  '산업',
  '활성',
  '활성방안',
  '방안',
  '연구',
  '책자',
  '3월',
  '23',
  '23일',
  '발행',
  '기계',
 

In [6]:
from apyori import apriori 

rules = apriori(news, min_support=0.3, min_confidence=0.2)
result = list(rules)

NameError: name 'news' is not defined

In [18]:
for r in result:
    print(r, end='\n\n')
# (lift=0.1 인 애들은 상관관계가 없으므로 빼고 확인한다.)

RelationRecord(items=frozenset({'10'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'10'}), confidence=0.3333333333333333, lift=1.0)])

RelationRecord(items=frozenset({'26'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'26'}), confidence=0.6, lift=1.0)])

RelationRecord(items=frozenset({'26일'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'26일'}), confidence=0.6, lift=1.0)])

RelationRecord(items=frozenset({'경우'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'경우'}), confidence=0.3, lift=1.0)])

RelationRecord(items=frozenset({'계획'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'계획'}), confidence=0.3, lift=1.0)])

RelationRecord(items=frozenset({'관련'}), support=0.7333333333333333, ordered_statistics=[OrderedStatistic(ite

In [20]:
result[3]

RelationRecord(items=frozenset({'경우'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'경우'}), confidence=0.3, lift=1.0)])

In [ ]:
import pandas as pd 
# 위의 결과를 데이터프레임으로 
result_df = pd.DataFrame(None, 
                         columns = ['lhs', 'rhs', 'support', 'confidence', 'lift'])
index = 0
for row in result:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ','.join(x for x in item[0]) # 데이터프레임에 담을 때에는 출력할 때처럼 []안에 담을 필요없다.
        rhs = ','.join(x.strip() for x in item[1])
        confidence = item[2]
        lift = item[3]
        if lift != 1:
            result_df.loc[index] = [lhs, rhs, support, confidence, lift]
            index += 1 
result_df

In [27]:
pd.options.display.max_rows = 644
result_df

,lhs,rhs,support,confidence,lift
0,26,26일,0.600000,1.000000,1.666667
1,26일,26,0.600000,1.000000,1.666667
2,26,관련,0.433333,0.722222,0.984848
3,관련,26,0.433333,0.590909,0.984848
4,26,기사,0.333333,0.555556,1.190476
5,기사,26,0.333333,0.714286,1.190476
6,26,기자,0.533333,0.888889,1.269841
7,기자,26,0.533333,0.761905,1.269841
8,26,사진,0.333333,0.555556,1.190476
9,사진,26,0.333333,0.714286,1.190476


In [30]:
result_df.loc[(result_df.lhs.str.contains('기자')) & result_df.rhs.str.contains('한국')].sort_values(by=['lift'], 
                                                                                                 ascending=False)

,lhs,rhs,support,confidence,lift
267,"관련,기자",한국,0.3,0.529412,1.134454
265,기자,"관련,한국",0.3,0.428571,1.071429
92,기자,한국,0.3,0.428571,0.918367


**연습하기**

In [4]:
import requests
from bs4 import BeautifulSoup
from konlpy.tag import Kkma

In [5]:
kkma = Kkma()
test_url = "https://search.daum.net/search?nil_suggest=btn&w=tot&DA=SBC&q=%EC%BD%94%EB%A1%9C%EB%82%98"
test_response = requests.get(test_url)
test_response
test_soup = BeautifulSoup(test_response.content, 'html.parser')
text_content = test_soup.select('.f_eb')

news = []
for text in text_content:
    news.append(list(filter(lambda word : len(word)>1 and word!='기자',
                           kkma.nouns(text.text))))

In [15]:
news_nouns = set()
news
for new in news:
    for n in new:
        news_nouns.add(n)
        
print('코로나 기사에 나오는 단어들: ', news_nouns)
print(len(news_nouns))

코로나 기사에 나오는 단어들:  {'캠페인', '감염증', '유행', '입원', '유의사항', '일상', '100', '메드', '우리', '포스팅', '질병관리청', '예방', '요즘', '대기', '지역콜센터', '12', '병원', '센터', '모하', '동반', '미취업자', '사망자', '강도', '안내', '대상', '집회', '최대', '데이터', '호흡기증상', '120', '상담', '특징', '주최', '4월', '의료기관', '인후통', '자치구', '과테말라', '바이러스', '지급', '200만', '-19', '자료', '2주', '관심', '뉴시스', '최저가', '정리', '뉴스', '지도', '안채원', '효과', '브리핑', '광화', '광고주의', '스감염증-19', '001', '지역국번', '중국', '20210423102842628', '뮤직', '존스', '기체', '차트', '검색어', '현황', '허락', '영화', '사항', '전리', '화재', '정부', '광화문', '문재인', '인후', '보관', '존스홉킨스대학교', '정치적', '스쿨', '7월', '일반', '돌파', '50', '피해', '실시간', '9월', '발생', '초고온', '식약처', '주의', '표면', '국외', '전염성', '필요', '동안', '서울', '우선', '육안', '수행', '지지율', '광고노출기준', '이온화', '대표', '단축', '38', '서울시', '5월', '이동', '최소', '위기', '태양', '계획', '악기', '6월', '폐렴', '일일확진자수가', '나눔캠페인', '지역', '정부대응발표', '현지', '생각', '개발', '불가리스', '의료', '스감', '스감염증', '행동', '냉동고', '정례', '증상', '동향', '국번', '마스크', '보건소', '방문자', '대량', '발열', '앵커', '무시무시', '31', '참석자', '공식', '지적', '19', '순서', '취업자', 

In [13]:
rules = apriori(news, 
                min_support=0.2, 
                min_confidence=0.2)
result = list(rules)
result

[RelationRecord(items=frozenset({'19'}), support=0.24324324324324326, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'19'}), confidence=0.24324324324324326, lift=1.0)]),
 RelationRecord(items=frozenset({'코로나'}), support=0.5405405405405406, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'코로나'}), confidence=0.5405405405405406, lift=1.0)]),
 RelationRecord(items=frozenset({'코로나19'}), support=0.24324324324324326, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'코로나19'}), confidence=0.24324324324324326, lift=1.0)]),
 RelationRecord(items=frozenset({'코로나', '19'}), support=0.24324324324324326, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'19', '코로나'}), confidence=0.24324324324324326, lift=1.0), OrderedStatistic(items_base=frozenset({'19'}), items_add=frozenset({'코로나'}), confidence=1.0, lift=1.8499999999999999), OrderedStatistic(items_base=frozenset({'코로나

In [16]:
from gensim.models import Word2Vec

In [17]:
model = Word2Vec(news,
                 window=5,
                 min_count=2,
                 workers=-1)

In [19]:
model.wv.most_similar("코로나")

[('120', 0.21883949637413025),
 ('호흡기', 0.21617144346237183),
 ('서울', 0.19543619453907013),
 ('앵커', 0.17216654121875763),
 ('기준', 0.1691875159740448),
 ('정부', 0.15161022543907166),
 ('코로나바이러스', 0.14176101982593536),
 ('보건소', 0.1403549313545227),
 ('접종', 0.13964074850082397),
 ('마스크', 0.1233743280172348)]